In [1]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 torchtext==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# Reload environment
exit()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.5 MB/s eta 0:12:31tcmalloc: large alloc 1147494400 bytes == 0x39154000 @  0x7f1e7bc4f615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:11:23tcmalloc: large alloc 1434370048 bytes == 0x7d7aa000 @  0x7f1e7bc4f615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████

In [3]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, TabularDataset, BucketIterator
import numpy as np
import spacy
import re
import random
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# python -m spacy download en
spacy_en = spacy.load("en_core_web_sm")


def tokenize(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


quote = Field(sequential=True, use_vocab=True, tokenize=tokenize, lower=True)
score = Field(sequential=False, use_vocab=False)

fields = {"quote": ("q", quote), "score": ("s", score)}

In [ ]:

train_data, test_data = TabularDataset.splits(
    path="/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess", train="train.json", test="test.json", format="json", fields=fields
)


NameError: ignored

In [ ]:
print(train_data[0].__dict__.keys())
print(train_data[0].__dict__.values())

dict_keys(['q', 's'])
dict_values([['you', 'must', 'own', 'everything', 'in', 'your', 'world', '.', 'there', 'is', 'no', 'one', 'else', 'to', 'blame', '.'], 1])


In [ ]:

  quote.build_vocab(train_data, max_size=10000, min_freq=2, vectors="glove.6B.100d")

  train_iterator, test_iterator = BucketIterator.splits(
      (train_data, test_data), batch_size=2, device=device
  )


.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|█████████▉| 399999/400000 [00:18<00:00, 22195.76it/s]


In [ ]:

for batch_idx, batch in enumerate(train_iterator):
        print(batch.q)
        # print(batch.s)

tensor([[14, 29],
        [25, 31],
        [ 7,  3],
        [ 5, 11],
        [10, 28],
        [15, 22],
        [21,  4],
        [ 3, 27],
        [ 7,  9],
        [ 5,  1],
        [32,  1],
        [30,  1],
        [ 8,  1],
        [17,  1],
        [ 4,  1],
        [13,  1],
        [ 6,  1],
        [ 2,  1]])
tensor([[35],
        [23],
        [26],
        [18],
        [19],
        [36],
        [34],
        [ 2],
        [33],
        [20],
        [24],
        [ 6],
        [16],
        [ 8],
        [12],
        [ 2]])


In [ ]:
!python -m spacy download de_core_news_sm

2022-10-02 01:42:00.238497: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 2.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [ ]:


spacy_eng = spacy.load("en_core_web_sm")
spacy_ger = spacy.load("de_core_news_sm")


def tokenize_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]


def tokenize_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]


In [ ]:

english = Field(sequential=True, use_vocab=True, tokenize=tokenize_eng, lower=True)
german = Field(sequential=True, use_vocab=True, tokenize=tokenize_ger, lower=True)

train_data, validation_data, test_data = Multi30k.splits(
    exts=(".de", ".en"), fields=(german, english)
)



downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:04<00:00, 254kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 92.4kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 87.8kB/s]


In [ ]:

english.build_vocab(train_data, max_size=10000, min_freq=1, vectors="glove.6B.100d")
german.build_vocab(train_data, max_size=10000, min_freq=1, vectors="glove.6B.100d")


In [ ]:
train_iterator, validation_iterator, test_iterator = BucketIterator.splits(
    (train_data, validation_data, test_data), batch_size=64, device="cpu"
)


In [ ]:

for batch in train_iterator:
    print(batch)

# string to integer (stoi)
print(f'Index of the word (the) is: {english.vocab.stoi["the"]}')

# print integer to string (itos)
print(f"Word of the index (1612) is: {english.vocab.itos[1612]}")
print(f"Word of the index (0) is: {english.vocab.itos[0]}")

In [ ]:
# 資料來源 https://statmt.org/wmt20/translation-task.html
# https://data.statmt.org/news-commentary/v15/

In [ ]:
tsv_file = open("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/news-commentary-v15.en-zh.tsv", encoding="utf8").read().split("\n")


In [ ]:
tsv_file

In [ ]:
my_raw_data = {"English" : [], "Chinese": []}

for line in tsv_file:
    sp_line = line.split('\t')
    my_raw_data['English'].append(sp_line[0])
    my_raw_data['Chinese'].append(sp_line[1])
# my_raw_data

IndexError: ignored

In [ ]:


df = pd.DataFrame(my_raw_data, columns=["English", "Chinese"])

# create train and test set
train, test = train_test_split(df, test_size=0.1)

# Get train, test data to json and csv format which can be read by torchtext
train.to_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/t3_train.json", orient="records", lines=True, force_ascii=False)
test.to_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/t3_test.json", orient="records", lines=True, force_ascii=False)

# train.to_csv("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/t3_train.csv", index=False)
# test.to_csv("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/t3_test.csv", index=False)

In [ ]:
# 處理null 數值

In [ ]:
import pandas as pd
train_df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/t3_train.json" , lines=True)
train_v2 = train_df[train_df['Chinese'].apply(lambda x : len(x)>1)]
display(train_v2.head())
train_v2.to_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/ml_translate_train.json", orient="records", lines=True, force_ascii=False)


test_df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/t3_test.json" , lines=True)
test_v2 = test_df[test_df['Chinese'].apply(lambda x : len(x)>1)]
display(test_df.head())
test_df.to_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess/ml_translate_test.json", orient="records", lines=True, force_ascii=False)

In [6]:
!python -m spacy download zh_core_web_sm

SyntaxError: ignored

In [ ]:


spacy_eng = spacy.load("en_core_web_sm")
spacy_zh = spacy.load("zh_core_web_sm")


def tokenize_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]


def tokenize_zh(text):
    return [tok.text for tok in spacy_zh.tokenizer(text)]


english = Field(sequential=True, use_vocab=True, tokenize=tokenize_eng, lower=True)
chinese = Field(sequential=True, use_vocab=True, tokenize=tokenize_zh, lower=True)

fields = {"English": ("eng", english), "Chinese": ("zh", chinese)}

train_data, test_data = TabularDataset.splits(
    path="/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess", train="t3_train.json", test="t3_test.json", format="json", fields=fields
)


In [ ]:

english.build_vocab(train_data, max_size=10000, min_freq=1, vectors="glove.6B.100d")
chinese.build_vocab(train_data, max_size=10000, min_freq=1, vectors="glove.6B.100d")

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data), batch_size=32, device="cuda"
)

for batch in train_iterator:
    print(batch)
    

# [IT 鐵人幫的資料](https://ithelp.ithome.com.tw/articles/10272027)


In [5]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_anki/'
lines = open(data_dir + 'cmn.txt' , encoding='utf-8').read().strip().split('\n')
trnslt_pairs = [[s for s in l.split('\t')] for l in lines ]
print ("Sample: " , trnslt_pairs[1000][0:2] )
print ("Total records:" , len(trnslt_pairs))

# Sample:  ['I want a book.', '我想要本书。']
# Total records: 29371

Sample:  ['I want a book.', '我想要本书。']
Total records: 29371


In [11]:
my_raw_data = {"English" : [], "Chinese": []}

for line in trnslt_pairs:
    my_raw_data['English'].append(line[0])
    my_raw_data['Chinese'].append(line[1])

q_key = 1003
print(my_raw_data["English"][q_key], my_raw_data["Chinese"][q_key])

I was at home. 我刚才在家。


In [13]:


df = pd.DataFrame(my_raw_data, columns=["English", "Chinese"])

# create train and test set
train, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train, test_size=0.08)


# Get train, test data to json and csv format which can be read by torchtext
train.to_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_anki/anki_train.json", orient="records", lines=True, force_ascii=False)
test.to_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_anki/anki_test.json", orient="records", lines=True, force_ascii=False)
val.to_json("/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_anki/anki_val.json", orient="records", lines=True, force_ascii=False)


In [ ]:
spacy_zh = spacy.load("zh_core_web_sm")
spacy_eng = spacy.load("en_core_web_sm")

def tokenize_eng(text):
    text = re.sub(r"([.!?])", r" \1", text)
    return [tok.text for tok in spacy_eng.tokenizer(text)]


def tokenize_zh(text):
    regex = re.compile(r'[^\u4e00-\u9fa5A-Za-z0-9]')
    text = regex.sub(' ', text)

    return [tok.text for tok in spacy_zh.tokenizer(text)]


english = Field(sequential=True, use_vocab=True, tokenize=tokenize_eng, lower=True)
chinese = Field(sequential=True, use_vocab=True, tokenize=tokenize_zh, lower=True)

fields = {"English": ("eng", english), "Chinese": ("zh", chinese)}

train_data, test_data = TabularDataset.splits(
    path="/content/drive/MyDrive/Colab Notebooks/ithome/torchtext_preprocess", train="ml_translate_train.json", test="ml_translate_test.json", format="json", fields=fields
)
